In [ ]:
pip install opencv-python
pip install keras
pip install opencv-python-headless

: 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import seaborn as sn
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from IPython.display import display, clear_output
from tkinter import filedialog
from tkinter import Tk

In [ ]:
def CreateModel():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (6,6), padding = 'same',activation='relu', input_shape=(363, 363, 3)))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32, (6,6), padding = 'same', activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32, (6,6), padding = 'same', activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    layers.Dropout(0.2)
    model.add(layers.Flatten())
    model.add(layers.Dense(units = 54, activation = 'relu'))
    model.add(layers.Dense(units = 10, activation = 'softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
def TrainModel(data_dir):
    image_size = (363, 363)
    batch_size = 128

    train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="both",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    class_names = test_ds.class_names



    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
    
    print(test_ds)
    
    epochs = 10
    history = model.fit(train_ds, epochs=epochs,
                        validation_data=(test_ds))
    
    return test_ds, history, epochs, class_names

In [ ]:
def LossHistory(history):
    # your code here
    acc = history.history['accuracy']
    acc_val = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(14, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label = 'Accuracy Score', color = 'blue')
    plt.plot(epochs_range, acc_val, label = 'Validated Accuracy Score', color = 'green')
    plt.legend()
    plt.title('Accuracy Score vs Validated Accuracy Score')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label = 'Loss', color = 'blue')
    plt.plot(epochs_range, val_loss, label = 'Validated Loss', color = 'green')
    plt.legend()
    plt.title('Loss Score vs Validated Loss Score')

    plt.show()

In [ ]:
def EvaluateModel(model, test_ds):
    # Get the image and label arrays for the first batch of validation data
    from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

    images = []
    labels = []
    for ims, lbs in test_ds:
        images = ims
        labels = lbs
    #print(f'Images = {images}: Labels = {labels}')

    
    # Use the model to predict the class
    predictions = model.predict(images)
    print(predictions)

    
    # The model returns a probability value for each class    
    # The one with the highest probability is the predicted class
    # The actual labels are hot encoded (e.g. [0 1 0], so get the one with the value 1
    predicted_classes = []
    for prd in predictions:
        predicted_classes.append(np.argmax(prd))

    #print(predicted_classes)

    # Plot the confusion matrix
    cm = confusion_matrix(labels, predicted_classes)
    ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_names).plot()

In [ ]:
def Predict(img):

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    identification = class_names[np.argmax(score)]
    
    return(identification)